# **Coding Project: Encryption in Python | Shreeja Das**
---

### **Description**
In this project, I wrote code to encrypt and decrypt a message of my choice using a key generated through QKD.

<br>

In [99]:
!pip install cirq
import cirq
import random

In [100]:
def QKD(num_bits):
    # Setup
    encode_gates = {0: cirq.I, 1: cirq.X}
    basis_gates = {'Z': cirq.I, 'X': cirq.H}

    qubits = cirq.NamedQubit.range(num_bits, prefix='q')

    # Alice Chooses Bits and Bases
    alice_key = random.choices([0, 1], k=num_bits)
    alice_bases = random.choices(['Z', 'X'], k=num_bits)

    # Alice Creates Qubits
    alice_circuit = cirq.Circuit()

    for bit in range(num_bits):
        encode_value = alice_key[bit]
        encode_gate = encode_gates[encode_value]

        basis_value = alice_bases[bit]
        basis_gate = basis_gates[basis_value]

        qubit = qubits[bit]
        alice_circuit.append(encode_gate(qubit))
        alice_circuit.append(basis_gate(qubit))

    # Bob chooses Bases
    bob_bases = random.choices(['Z', 'X'], k=num_bits)

    bob_circuit = cirq.Circuit()

    for bit in range(num_bits):
        basis_value = bob_bases[bit]
        basis_gate = basis_gates[basis_value]

        qubit = qubits[bit]
        bob_circuit.append(basis_gate(qubit))

    # Bob Measures Qubits
    bob_circuit.append(cirq.measure(qubits, key='bob key'))

    # Bob Creates a Key
    bb84_circuit = alice_circuit + bob_circuit

    sim = cirq.Simulator()
    results = sim.run(bb84_circuit)
    bob_key = results.measurements['bob key'][0]

    final_alice_key = []
    final_bob_key = []

    # Compare Bases
    for bit in range(num_bits):
        if alice_bases[bit] == bob_bases[bit]:
            final_alice_key.append(alice_key[bit])
            final_bob_key.append(bob_key[bit])

    # Compare Half their Bits
    num_bits_to_compare = int(len(final_alice_key) * .5)
    if final_alice_key[0:num_bits_to_compare] == final_bob_key[0:num_bits_to_compare]:
        final_alice_key = final_alice_key[num_bits_to_compare:]
        final_bob_key = final_bob_key[num_bits_to_compare:]
        return final_alice_key, final_bob_key  # Corrected: Return the keys if the condition is met
    else:
        print('\n\nEve was listening, we need to use a different channel!')
        return [], []  # Corrected: Return empty lists if the condition is not met


### **Step 1**
Creating data I want to send!

In [101]:
unencrypted_string = "Shhh super secret"

### **Step 2**
Use the `QKD` function, defined above, to create a key for my data.

In [102]:
num_bits = len(unencrypted_string) * 8
alice_key, bob_key = QKD(num_bits)
print("Alice key: ", alice_key)
print("\nBob key: ", bob_key)


Alice key:  [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0]

Bob key:  [1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0]


### **Step 3**
---
Here, I created a function that can now ecrypt my message using my key.

In [103]:
def encrypt_message(message, key):
    encrypted_message = ""
    for i, char in enumerate(message):
        # Convert char to ASCII code, perform XOR with key bit, convert back to char
        new_char_code = ord(char) ^ key[i % len(key)]
        encrypted_message += chr(new_char_code)
    return encrypted_message

### **Step 4**

Now, I created a function that can decrypt my message through the key.

In [104]:

def decrypt_message(encrypted_message, key):
    decrypted_message = ""
    for i, char in enumerate(encrypted_message):
        # Reverse XOR operation from encryption
        new_char_code = ord(char) ^ key[i % len(key)]
        decrypted_message += chr(new_char_code)
    return decrypted_message


### **Step 5**
---
Putting it all together and testing it out!!

In [105]:
if alice_key and bob_key:
    encrypted_message = encrypt_message(unencrypted_string, alice_key)
    decrypted_message = decrypt_message(encrypted_message, bob_key)

    print("Original message:", unencrypted_string)
    print("Encrypted Message:", encrypted_message)
    print("Decrypted message:", decrypted_message)

else:
    print("Failed to generate a secure key. Try running the program again.")


Original message: Shhh super secret
Encrypted Message: Rihi!stqes secset
Decrypted message: Shhh super secret


# End of Notebook

---
© 2024 The Coding School, All rights reserved